### Critérios de segmentação

Para segmentação dos usuários, serão usados os seguintes critérios:
| Segmento                              | Critério técnico (coluna e regra)                                                  | Racional                                                                                       |
| ------------------------------------- | ---------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------- |
| **Perfil de consumo por restaurante** | `price_range`:<br>1 → Baixo valor<br>2 → Intermediário<br>3 ou mais → Alto valor   | Identifica o tipo de restaurante que o usuário consome — restaurantes baratos ou premium       |
| **Localização do pedido (estado)**    | `delivery_address_state` agrupado como coluna categórica                           | Detecta diferenças de comportamento por região geográfica (ex: SP vs. RJ vs. outros)           |
| **Ticket médio do usuário**           | `avg_ticket_per_user`:<br>< R\$ 40 → Baixo<br>R\$ 40–60 → Médio<br>> R\$ 60 → Alto | Mede a propensão do usuário a gastar mais por pedido — pode reagir de forma diferente ao cupom |
| **Frequência de pedidos do usuário**  | `orders_per_user`:<br>1 → Baixa<br>2–4 → Média<br>5+ → Alta                        | Indica engajamento prévio. O cupom pode ter mais impacto em quem pede pouco                    |


In [0]:
import pyspark.sql.functions as f

from constants import SILVER_LAYER_PATH

import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu, shapiro
from statsmodels.stats.proportion import proportions_ztest

import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
def plot_segment_distribution(
    df, column, title=None, order=None, figsize=(6, 4), rotation=15
):
    """
    Gera um gráfico de barras (countplot) para um segmento categórico.

    Parâmetros:
    - df: DataFrame Pandas
    - column: nome da coluna categórica
    - title: título do gráfico
    - order: ordem dos rótulos no eixo X (opcional)
    - figsize: tamanho do gráfico
    - rotation: rotação do eixo X
    """
    plt.figure(figsize=figsize)
    sns.countplot(x=column, data=df, order=order)
    plt.title(title or f"Distribuição de {column}")
    plt.ylabel("Número de usuários")
    plt.xlabel("Segmento")
    plt.xticks(rotation=rotation)
    plt.tight_layout()
    plt.show()

def plot_multiple_segment_distributions(
    df,
    columns_info,
    figsize=(18, 5),
    rotation=15,
    label_fontsize=9,
    margin_factor=1.20
):
    """
    Plota múltiplos gráficos de barras com proporções (%) lado a lado.

    Parâmetros:
    - df: DataFrame Pandas
    - columns_info: lista de dicionários com:
        - 'column': nome da coluna a plotar
        - 'title': título do gráfico
        - 'order': ordem opcional dos rótulos
    - figsize: tamanho total da figura
    - rotation: rotação dos rótulos no eixo X
    """
    n_cols = len(columns_info)
    fig, axes = plt.subplots(1, n_cols, figsize=figsize)

    for i, info in enumerate(columns_info):
        col = info['column']
        title = info.get('title', f"Distribuição de {col}")
        order = info.get('order')

        # Calcula proporções
        counts = df[col].value_counts(normalize=True)
        if order:
            counts = counts.reindex(order)
        counts = counts.dropna()

        sns.barplot(x=counts.index, y=counts.values * 100, ax=axes[i])
        axes[i].set_title(title)
        axes[i].set_ylabel("Proporção (%)")
        axes[i].set_xlabel("")
        axes[i].tick_params(axis='x', rotation=rotation)

        # Define limite do eixo Y com margem para rótulo
        max_y = counts.values.max() * 100
        axes[i].set_ylim(0, max_y * margin_factor)

        # Adiciona rótulos
        for bar, value in zip(axes[i].patches, counts.values):
            height = bar.get_height()
            axes[i].text(
                bar.get_x() + bar.get_width() / 2,
                height + max_y * 0.02,
                f"{value*100:.1f}%",
                ha='center',
                va='bottom',
                fontsize=label_fontsize
            )

    plt.subplots_adjust(top=0.85, bottom=0.15)
    plt.show()

def plot_segment_comparison_by_group(
    df,
    segment_col,
    group_col='is_target',
    title=None,
    order=None,
    labels={True: 'Target', False: 'Controle'},
    figsize=(6, 5),
    rotation=15,
    show_stats=True
):
    """
    Compara proporções de um segmento entre dois grupos (ex: target vs controle).

    Parâmetros:
    - df: DataFrame Pandas
    - segment_col: nome da coluna com a segmentação (ex: 'segment_ticket')
    - group_col: coluna binária de grupos (default: 'is_target')
    - title: título do gráfico
    - order: ordem das categorias da segmentação (opcional)
    - labels: dicionário com os nomes dos grupos (ex: {True: 'Target', False: 'Controle'})
    """
    contingency = pd.crosstab(df[segment_col], df[group_col])
    chi2, p, dof, expected = chi2_contingency(contingency)

    if show_stats:
        print(f"📊 Teste Qui-Quadrado entre {segment_col} e {group_col}")
        print(f"Chi² = {chi2:.2f}, p-value = {p:.5f}")
        if p < 0.05:
            print("✅ Diferença estatisticamente significativa (p < 0.05)")
        else:
            print("❌ Diferença NÃO significativa (p >= 0.05)")

    # Proporções por grupo
    prop_df = (
        df.groupby([group_col, segment_col])
        .size()
        .groupby(level=0, group_keys=False)
        .apply(lambda x: x / x.sum())
        .reset_index(name='proportion')
    )

    # Map labels
    prop_df[group_col] = prop_df[group_col].map(labels)

    # Plot
    plt.figure(figsize=figsize)
    sns.barplot(
        data=prop_df,
        x=segment_col,
        y='proportion',
        hue=group_col,
        order=order
    )

    plt.title(title or f"{segment_col} por grupo (p = {p:.3f})")
    plt.ylabel("Proporção (%)")
    plt.xlabel("")
    plt.xticks(rotation=rotation)
    plt.ylim(0, prop_df['proportion'].max() * 1.2)

    # Add labels on bars
    for bar in plt.gca().patches:
        height = bar.get_height()
        plt.gca().text(
            bar.get_x() + bar.get_width() / 2,
            height + 0.01,
            f"{height*100:.1f}%",
            ha='center',
            va='bottom',
            fontsize=9
        )

    plt.legend(title="Grupo")
    plt.tight_layout()
    plt.show()

def plot_kpi_boxplot_by_segment_and_group(
    df,
    segment_col,
    group_col,
    kpi_col,
    order=None,
    group_labels={True: 'Target', False: 'Controle'},
    title=None,
    figsize=(8, 5)
):
    """
    Gera um boxplot do KPI por grupo dentro de cada segmento.
    """
    df_plot = df.copy()
    df_plot[group_col] = df_plot[group_col].map(group_labels)

    plt.figure(figsize=figsize)
    sns.boxplot(
        data=df_plot,
        x=segment_col,
        y=kpi_col,
        hue=group_col,
        order=order,
        showfliers=False 
    )

    plt.title(title or f"{kpi_col} por grupo dentro de {segment_col}")
    plt.ylabel(kpi_col.replace("_", " ").title())
    plt.xlabel("")
    plt.xticks(rotation=15)
    plt.legend(title="Grupo")
    plt.tight_layout()
    plt.show()

def test_kpi_significance_by_segment(
    df,
    segment_col,
    group_col,
    kpi_col,
    group_labels={True: "Target", False: "Controle"},
    normality_threshold=0.05,
    min_sample_for_ttest=30
):
    """
    Compara um KPI entre grupos dentro de cada segmento, escolhendo automaticamente entre t-test e Mann-Whitney.
    """
    results = []

    for segment in sorted(df[segment_col].dropna().unique()):
        data = df[df[segment_col] == segment]
        g1 = data[data[group_col] == True][kpi_col].dropna()
        g2 = data[data[group_col] == False][kpi_col].dropna()

        n1, n2 = len(g1), len(g2)
        mean1, mean2 = g1.mean(), g2.mean()
        pct_diff = 100 * (mean1 - mean2) / mean2 if mean2 else None

        # Definir teste
        test_used = None
        p_value = None

        if n1 < 5 or n2 < 5:
            test_used = "amostra insuficiente"
        elif n1 >= min_sample_for_ttest and n2 >= min_sample_for_ttest:
            stat, p_value = ttest_ind(g1, g2, equal_var=False)
            test_used = "t-test"
        else:
            # Verificar normalidade
            p1 = shapiro(g1)[1] if n1 >= 3 and n1 <= 5000 else 0  # fora do intervalo, assume não normal
            p2 = shapiro(g2)[1] if n2 >= 3 and n2 <= 5000 else 0
            if p1 > normality_threshold and p2 > normality_threshold:
                stat, p_value = ttest_ind(g1, g2, equal_var=False)
                test_used = "t-test"
            else:
                stat, p_value = mannwhitneyu(g1, g2, alternative="two-sided")
                test_used = "mann-whitney"

        results.append({
            "Segmento": segment,
            "n_target": n1,
            "n_control": n2,
            "Media Target": mean1,
            "Media Controle": mean2,
            "Diferença (%)": f"{pct_diff:+.1f}%" if pct_diff is not None else "N/A",
            "p-valor": f"{p_value:.5f}" if p_value is not None else "N/A",
            "Significativo": "✅" if p_value is not None and p_value < 0.05 else "❌",
            "Teste usado": test_used
        })

    return pd.DataFrame(results)

In [0]:
abt_final = spark.table(f"{SILVER_LAYER_PATH}.abt_final")
abt_final.printSchema()

In [0]:
user_metrics_df = (
    abt_final.groupBy("customer_id", "is_target")
    .agg(
        f.countDistinct("order_id").alias("orders_per_user"),
        f.sum("order_total_amount_capped").alias("total_spent_per_user"),
        f.sum("total_items_quantity").alias("total_items"),
        f.sum("total_items_discount").alias("total_discount"),
        f.min("days_since_signup").alias("days_since_signup"),
        f.first("price_range").alias("price_range"),
        f.first("delivery_address_state").alias("delivery_address_state")
    )
    .withColumn("avg_ticket_per_user", f.col("total_spent_per_user") / f.col("orders_per_user"))
    .withColumn("avg_items_per_order", f.col("total_items") / f.col("orders_per_user"))
    .withColumn("avg_discount_per_order", f.col("total_discount") / f.col("orders_per_user"))
    .withColumn("is_retained", f.when(f.col("orders_per_user") >= 2, 1).otherwise(0))
)

user_metrics_df.printSchema()

In [0]:
# frequeência de pedidos do usuário (engajamento)
user_metrics_df = user_metrics_df.withColumn(
    "segment_frequency",
    f.when(f.col("orders_per_user") == 1, "Baixa frequência")
    .when((f.col("orders_per_user") >= 2) & (f.col("orders_per_user") <= 4), "Média frequência")
    .otherwise("Alta frequência")
)

# ticket médio do usuário
user_metrics_df = user_metrics_df.withColumn(
    "segment_ticket",
    f.when(f.col("avg_ticket_per_user") < 40, "Baixo ticket")
    .when((f.col("avg_ticket_per_user") >= 40) & (f.col("avg_ticket_per_user") <= 60), "Ticket médio")
    .otherwise("Alto ticket")
)

# perfil do restaurante
user_metrics_df = user_metrics_df.withColumn(
    "segment_restaurant",
    f.when(f.col("price_range") == 1, "Baixo valor")
    .when(f.col("price_range") == 2, "Valor intermediário")
    .otherwise("Alto valor")
)

# localização do pedido
user_metrics_df = user_metrics_df.withColumnRenamed("delivery_address_state", "segment_state")

In [0]:
segments_pdf = user_metrics_df.select(
    "customer_id",  
    "is_target",  
    "segment_frequency",
    "segment_ticket",
    "segment_restaurant",
    "segment_state"
).toPandas()

In [0]:
plot_multiple_segment_distributions(
    df=segments_pdf,
    columns_info=[
        {
            "column": "segment_frequency",
            "title": "Frequência de Uso",
            "order": ["Baixa frequência", "Média frequência", "Alta frequência"]
        },
        {
            "column": "segment_ticket",
            "title": "Ticket Médio",
            "order": ["Baixo ticket", "Ticket médio", "Alto ticket"]
        },
        {
            "column": "segment_restaurant",
            "title": "Perfil de Restaurante",
            "order": ["Baixo valor", "Valor intermediário", "Alto valor"]
        }
    ]
)

In [0]:
plot_segment_distribution(
    df=segments_pdf,
    column="segment_state",
    title="Distribuição por Estado",
    rotation=45,
    figsize=(10, 4)
)

### Frequência de pedidos

In [0]:
user_metrics_pdf = user_metrics_df.toPandas()

In [0]:
plot_segment_comparison_by_group(
    df=segments_pdf,
    segment_col="segment_frequency",
    title="Distribuição de Frequência por Grupo",
    order=["Baixa frequência", "Média frequência", "Alta frequência"]
)

O teste de qui-quadrado avalia se há associação entre duas variáveis categóricas — neste caso, entre o grupo do experimento (`is_target`) e a frequência de pedidos dos usuários (`segment_frequency`).

Como o p-value é muito pequeno (< 0.05), a conclusão é:
* Existe uma diferença real na distribuição de frequência entre os grupos target e controle.
Ou seja, a campanha não foi aleatoriamente balanceada entre perfis de usuários com frequência baixa, média ou alta.
* Um bom teste A/B deve ter distribuição semelhante de perfis entre os grupos, para isolar o efeito do tratamento.
* Como houve desequilíbrio, é possível que parte do impacto observado nos KPIs venha do perfil de uso dos usuários, e não do cupom em si.

O que podemos fazer:
1. Controlar os efeitos de `segment_frequency` na análise dos KPIs — ex: comparar KPIs dentro de cada faixa de frequência
2. Considerar reanalisar a alocação dos grupos (se estiver rodando em produção)
3. Usar modelos ajustados ou segmentações para entender o impacto real



In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_frequency",
    group_col="is_target",
    kpi_col="total_spent_per_user"
)

display(df_test_result)

**Alta frequência**:
Já são usuários muito engajados — o cupom não foi suficiente para aumentar o gasto.

**Baixa frequência**:
A campanha não reativou usuários menos engajados de forma significativa.

**Média frequência**:
Leve queda no gasto, mas não significativa.

CONCLUSÃO:
A campanha não teve impacto significativo no total gasto por usuário, mesmo considerando o perfil de frequência.

In [0]:
plot_kpi_boxplot_by_segment_and_group(
    df=user_metrics_pdf,
    segment_col="segment_frequency",
    group_col="is_target",
    kpi_col="total_spent_per_user",
    order=["Baixa frequência", "Média frequência", "Alta frequência"],
    title="Distribuição do gasto total por grupo dentro de cada segmento"
)

In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_frequency",
    group_col="is_target",
    kpi_col="orders_per_user"
)

display(df_test_result)

**Alta frequência**:
Esses usuários já pedem muito, possivelmente, o cupom não alterou o hábito deles.

**Baixa frequência**:
Impossível calcular p-valor porque não há variação: todos pediram apenas 1 vez.
Isso sugere que a campanha não conseguiu estimular um segundo pedido neste grupo.

* Insight importante: o cupom não gerou recorrência nos menos engajados.

**Média frequência**:
A campanha não alterou a frequência de pedidos neste grupo também.

CONCLUSÃO:
1. O cupom não teve impacto significativo no número de pedidos por usuário, em nenhum segmento de frequência.

2. Mesmo onde havia espaço para crescimento (baixa e média frequência), o número de pedidos não se alterou.

3. É possível que o cupom tenha sido usado apenas em um único pedido e não tenha incentivado recorrência.

In [0]:
plot_kpi_boxplot_by_segment_and_group(
    df=user_metrics_pdf,
    segment_col="segment_frequency",
    group_col="is_target",
    kpi_col="orders_per_user",
    order=["Baixa frequência", "Média frequência", "Alta frequência"],
    title="Distribuição dos pedidos por usuário por grupo dentro de cada segmento"
)

In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_frequency",
    group_col="is_target",
    kpi_col="is_retained"
)

display(df_test_result)

O KPI `is_retained` não apresentou variabilidade suficiente dentro dos segmentos para que um teste de significância fosse válido.
Isso impede a detecção de impacto da campanha sobre retenção dentro desses grupos.

### Ticket Médio

In [0]:
plot_segment_comparison_by_group(
    df=segments_pdf,
    segment_col="segment_ticket",
    title="Distribuição de Ticket médio por Grupo",
    order=["Baixo ticket", "Ticket médio", "Alto ticket"]
)

In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_ticket",
    group_col="is_target",
    kpi_col="total_spent_per_user"
)

display(df_test_result)

O cupom teve impacto significativo no aumento do total gasto por usuário em todos os segmentos de ticket médio.

O efeito foi:
Consistente entre os grupos (entre +12% e +14%) e estatisticamente forte (p < 0.00001 em todos os casos)

**Baixo ticket**:
Maior impacto percentual (+14,3%)
Isso sugere que usuários que normalmente gastam pouco foram estimulados a comprar mais

**Ticket médio**:
Também apresentou ganho expressivo de +12,6%
Esse grupo pode estar mais propenso a aproveitar cupons sem mudar tanto seu comportamento de compra

**Alto ticket**:
Mesmo usuários que já gastavam muito gastaram ainda mais com o cupom
Isso indica que a campanha também funcionou para clientes de alto valor

CONCLUSÃO:
O cupom funcionou bem em todos os perfis de usuários com diferentes níveis de gasto médio.
Estratégias futuras podem incluir cupons com valor mínimo proporcional ao perfil de ticket médio para otimizar ROI.

In [0]:
plot_kpi_boxplot_by_segment_and_group(
    df=user_metrics_pdf,
    segment_col="segment_ticket",
    group_col="is_target",
    kpi_col="total_spent_per_user",
    order=["Baixo ticket", "Ticket médio", "Alto ticket"],
    title="Distribuição do gasto total por grupo dentro de cada segmento"
)

In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_ticket",
    group_col="is_target",
    kpi_col="orders_per_user"
)

display(df_test_result)

O cupom resultou em um número significativamente maior de pedidos por usuário em todos os segmentos de ticket médio.
Isso reforça que o aumento do gasto total se deve principalmente ao aumento na frequência de pedidos, não no valor por pedido

**Baixo ticket:**
Maior crescimento de frequência de pedidos (+13,7%). Mostra que o cupom engajou até os usuários com perfil de gasto menor, que voltaram para fazer novos pedidos

**Ticket médio:**
Ganho de +12,7% comportamento muito próximo ao dos outros segmentos. Indica que o cupom teve poder universal, sem restrição ao perfil de gasto

**Alto ticket:**
Também aumento claro de +13,3%. Mesmo os usuários que já pedem de forma robusta foram estimulados a pedir mais vezes

**Relação com resultado anterior (total_spent_per_user)**:
O crescimento de gasto está fortemente ligado à maior frequência de pedidos. Isso confirma o comportamento esperado: o cupom incentivou recompra

In [0]:
plot_kpi_boxplot_by_segment_and_group(
    df=user_metrics_pdf,
    segment_col="segment_ticket",
    group_col="is_target",
    kpi_col="orders_per_user",
    order=["Baixo ticket", "Ticket médio", "Alto ticket"],
    title="Distribuição dos pedidos por usuário por grupo dentro de cada segmento"
)

In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_ticket",
    group_col="is_target",
    kpi_col="is_retained"
)

display(df_test_result)

A campanha de cupom teve impacto claro e positivo na retenção dos usuários, em todas as faixas de ticket médio.
O efeito é forte e consistente, com aumento da taxa de retenção entre 18% e 22,5% e o p-valor muito pequeno (< 0.00001) indica que a probabilidade de isso ser ao acaso é praticamente nula.

**Baixo ticket:**
A maior diferença percentual (+22,5% de retenção), ou seja, o cupom foi especialmente eficaz para trazer de volta usuários com menor poder de compra

**Ticket médio:**
Ligeiramente menor (+18,2% de retenção), mas ainda muito relevante, reforçando que usuários “intermediários” também responderam positivamente à campanha

**Alto ticket:**
Mostra que o cupom também fortalece o vínculo com clientes de alto valor (+21,8% de retenção)

### Perfil restaurante

In [0]:
plot_segment_comparison_by_group(
    df=segments_pdf,
    segment_col="segment_restaurant",
    title="Distribuição de perfil do restaurante por Grupo",
    order=["Baixo valor", "Valor intermediário", "Alto valor"]
)

A distribuição dos perfis de restaurante entre os grupos target (que receberam cupom) e controle não é significativamente diferente.
Os usuários dos dois grupos têm distribuição semelhante nos tipos de restaurante que costumam consumir. Isso indica que a randomização foi bem feita para essa variável.

Como os grupos estão balanceados, qualquer diferença nos KPIs dentro dos perfis de restaurante pode ser atribuída com mais confiança à campanha de cupom, e não a viés de alocação.

In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_restaurant",
    group_col="is_target",
    kpi_col="total_spent_per_user"
)

display(df_test_result)

O cupom aumentou significativamente o total gasto por usuário em todos os perfis de restaurante — com impacto entre +13% e +14%, altamente significativo.

**Baixo valor:**
O cupom foi eficaz mesmo para usuários que costumam consumir de restaurantes mais baratos

**Valor intermediário:**
Reflete que esse público pode estar mais sensível ao benefício do cupom sem comprometer a margem

**Alto valor:**
Mostra que mesmo usuários com ticket mais elevado ampliaram seu consumo

CONCLUSÃO: A campanha funcionou de forma consistente para todos os perfis de restaurante, ampliando o total gasto por usuário com impacto relevante e estatisticamente robusto.

In [0]:
plot_kpi_boxplot_by_segment_and_group(
    df=user_metrics_pdf,
    segment_col="segment_restaurant",
    group_col="is_target",
    kpi_col="total_spent_per_user",
    order=["Baixo valor", "Valor intermediário", "Alto valor"],
    title="Distribuição do gasto total por grupo dentro de cada segmento"
)

In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_restaurant",
    group_col="is_target",
    kpi_col="orders_per_user"
)

display(df_test_result)

O cupom levou os usuários a fazerem mais pedidos independentemente do perfil de restaurante em que costumam consumir. A diferença foi uniforme, com crescimento de ~13% em todos os segmentos.

Isso reforça que o aumento no total gasto vem principalmente da maior frequência de compra, e não de aumento no ticket médio.

**Baixo valor:**
Público sensível a preço, mas o cupom foi um forte incentivo à recompra

**Valor intermediário:**
Mostra um comportamento equilibrado e previsível. Pode ser um dos segmentos com maior elasticidade ao incentivo

**Alto valor:**
O cupom é eficaz mesmo para quem já tem alta propensão de consumo

**Coerência com total_spent_per_user:**
O aumento em pedidos por usuário se reflete diretamente no maior gasto total

Isso confirma a coerência interna dos KPIs

**Conclusão**:
A campanha aumentou a frequência de pedidos em todos os perfis de restaurante, mantendo consistência de impacto com robustez estatística.

In [0]:
plot_kpi_boxplot_by_segment_and_group(
    df=user_metrics_pdf,
    segment_col="segment_restaurant",
    group_col="is_target",
    kpi_col="orders_per_user",
    order=["Baixo valor", "Valor intermediário", "Alto valor"],
    title="Distribuição dos pedidos por usuário por grupo dentro de cada segmento"
)

In [0]:
df_test_result = test_kpi_significance_by_segment(
    df=user_metrics_pdf,
    segment_col="segment_restaurant",
    group_col="is_target",
    kpi_col="is_retained"
)

display(df_test_result)

A campanha de cupom teve um impacto positivo altamente significativo na retenção dos usuários, independentemente do perfil de restaurante.
O aumento na proporção de usuários retidos foi de ~21% em todos os segmentos

O p-valor praticamente nulo indica que a chance desse efeito ser aleatório é desprezível.

**Baixo valor**
Mostra que o cupom é muito eficaz para reter consumidores mais sensíveis ao preço

**Valor intermediário**
Reforça que esse público responde bem a incentivos e pode manter comportamento de longo prazo

**Alto valor**
Usuários com alto valor de pedido foram mais fidelizados, o que aumenta o LTV (Lifetime Value)

**Coerência com demais KPIs:**
Esses resultados estão completamente alinhados com os de:

`total_spent_per_user` e `orders_per_user`

Ou seja, a campanha não apenas aumentou o consumo, mas também estimulou comportamentos de longo prazo

CONCLUSÃO:
A campanha de cupom teve impacto positivo, consistente e significativo em todos KPIs
E isso ocorreu de forma homogênea em todos os perfis de restaurante (baixo valor, intermediário e alto valor).

### Localidade

In [0]:
def build_state_analysis_table(df, state_col="segment_state"):
    states = df[state_col].dropna().unique()
    result_rows = []

    for state in sorted(states):
        subset = df[df[state_col] == state]

        # Gasto total
        tgt_spent = subset[subset["is_target"] == True]["total_spent_per_user"].dropna()
        ctl_spent = subset[subset["is_target"] == False]["total_spent_per_user"].dropna()
        mean_tgt_spent = tgt_spent.mean()
        mean_ctl_spent = ctl_spent.mean()
        p_spent = None
        spent_diff_pct = None
        if len(tgt_spent) >= 5 and len(ctl_spent) >= 5:
            stat_spent, p_spent = ttest_ind(tgt_spent, ctl_spent, equal_var=False)
            if mean_ctl_spent != 0:
                spent_diff_pct = 100 * (mean_tgt_spent - mean_ctl_spent) / mean_ctl_spent

        # Pedidos por usuário
        tgt_orders = subset[subset["is_target"] == True]["orders_per_user"].dropna()
        ctl_orders = subset[subset["is_target"] == False]["orders_per_user"].dropna()
        mean_tgt_orders = tgt_orders.mean()
        mean_ctl_orders = ctl_orders.mean()
        p_orders = None
        orders_diff_pct = None
        if len(tgt_orders) >= 5 and len(ctl_orders) >= 5:
            stat_orders, p_orders = ttest_ind(tgt_orders, ctl_orders, equal_var=False)
            if mean_ctl_orders != 0:
                orders_diff_pct = 100 * (mean_tgt_orders - mean_ctl_orders) / mean_ctl_orders

        # Retenção
        tgt_ret = subset[subset["is_target"] == True]["is_retained"].dropna()
        ctl_ret = subset[subset["is_target"] == False]["is_retained"].dropna()
        p_ret = None
        ret_diff_pct = None
        rate_tgt = tgt_ret.mean() if len(tgt_ret) > 0 else None
        rate_ctl = ctl_ret.mean() if len(ctl_ret) > 0 else None
        if len(tgt_ret) >= 30 and len(ctl_ret) >= 30:
            count = [tgt_ret.sum(), ctl_ret.sum()]
            nobs = [len(tgt_ret), len(ctl_ret)]
            stat_ret, p_ret = proportions_ztest(count, nobs)
            if rate_ctl != 0:
                ret_diff_pct = 100 * (rate_tgt - rate_ctl) / rate_ctl

        # Interpretações
        def interpret(p, pct, tipo):
            if p is None:
                return f"Dados insuficientes para {tipo}"
            if p < 0.05:
                if pct > 0:
                    return f"A campanha aumentou o {tipo.lower()} significativamente."
                else:
                    return f"O {tipo.lower()} caiu no grupo com cupom — possível efeito negativo."
            else:
                return f"Sem impacto significativo no {tipo.lower()}."

        def recommend(p_gasto, pct_gasto, p_retencao, pct_retencao, p_orders, pct_orders):
            if (
                (p_gasto is not None and p_gasto < 0.05 and pct_gasto > 5) or
                (p_retencao is not None and p_retencao < 0.05 and pct_retencao > 5) or
                (p_orders is not None and p_orders < 0.05 and pct_orders > 5)
            ):
                return "Ampliar a ação nesse estado."
            elif (p_gasto is not None and p_gasto >= 0.05) and \
                 (p_retencao is not None and p_retencao >= 0.05) and \
                 (p_orders is not None and p_orders >= 0.05):
                return "Testar alternativas (cupom menos atrativo, frete, etc)."
            else:
                return "Manter com ajustes ou reavaliar."

        result_rows.append({
            "Estado": state,
            "Δ Gasto (%)": f"{spent_diff_pct:+.1f}%" if spent_diff_pct is not None else "N/A",
            "p-valor Gasto": f"{p_spent:.5f}" if p_spent is not None else "N/A",
            "Significativo Gasto": "✅" if p_spent is not None and p_spent < 0.05 else "❌",
            "Δ Pedidos (%)": f"{orders_diff_pct:+.1f}%" if orders_diff_pct is not None else "N/A",
            "p-valor Pedidos": f"{p_orders:.5f}" if p_orders is not None else "N/A",
            "Significativo Pedidos": "✅" if p_orders is not None and p_orders < 0.05 else "❌",
            "Δ Retenção (%)": f"{ret_diff_pct:+.1f}%" if ret_diff_pct is not None else "N/A",
            "p-valor Retenção": f"{p_ret:.5f}" if p_ret is not None else "N/A",
            "Significativo Retenção": "✅" if p_ret is not None and p_ret < 0.05 else "❌",
            "Interpretação Gasto": interpret(p_spent, spent_diff_pct, "Gasto"),
            "Interpretação Pedidos": interpret(p_orders, orders_diff_pct, "Pedidos"),
            "Interpretação Retenção": interpret(p_ret, ret_diff_pct, "Retenção"),
            "Recomendação Final": recommend(p_spent, spent_diff_pct, p_ret, ret_diff_pct, p_orders, orders_diff_pct)
        })

    return pd.DataFrame(result_rows)

In [0]:
segment_state_table = build_state_analysis_table(user_metrics_pdf)

pd.set_option("display.max_colwidth", None)
display(segment_state_table)

### Análise por Segmentação
A análise por segmentações permitiu avaliar a efetividade da campanha de cupom em diferentes perfis de usuários e contextos de consumo. Foram exploradas segmentações com base na frequência de pedidos, no ticket médio e no perfil de restaurante consumido, permitindo identificar como o impacto da campanha varia (ou não) entre os grupos.

#### 1. Segmentação por frequência de pedidos (segment_frequency)
Os usuários foram agrupados em baixa, média e alta frequência de pedidos.

Entretanto, por construção, os grupos apresentaram pouca ou nenhuma variabilidade nos KPIs relacionados à frequência e retenção, tornando as análises estatísticas inconclusivas ou inválidas.

A segmentação revelou que o cupom não teve impacto em aumentar a frequência de pedidos entre os usuários de baixa frequência, o que indica que a campanha pode não ter sido suficiente para gerar recorrência nos menos engajados.

Como esperado, usuários de alta frequência já eram retidos, o que torna o impacto incremental mais difícil de observar nesse grupo.

##### ✅ Insight 
O cupom não alterou a frequência de quem menos pedia — talvez seja necessário um incentivo mais agressivo ou de outro tipo para esse público.

#### 2. Segmentação por ticket médio (segment_ticket)
Os usuários foram classificados em faixas de baixo, médio e alto ticket.

Os resultados foram consistentes e altamente positivos em todas as faixas

O cupom funcionou em todos os níveis de gasto, desde os consumidores mais econômicos até os de maior poder aquisitivo.

A retenção foi especialmente impactada, com ganhos acima de 18% em todos os segmentos, mostrando que o benefício gerou engajamento duradouro.

##### ✅ Insight 
A campanha foi eficaz em todos os perfis de consumo por valor, com impacto relevante no hábito de compra e fidelização.

#### 3. Segmentação por perfil de restaurante (segment_restaurant)
A distribuição dos grupos (target vs. controle) foi estatisticamente equilibrada (p = 0.106), o que confere credibilidade à análise dos efeitos por segmento.

Mais uma vez, o impacto do cupom foi forte e uniforme.

O cupom foi altamente eficaz independentemente do tipo de restaurante consumido — funcionando tanto para quem consome restaurantes mais populares quanto para quem consome restaurantes de ticket mais elevado.

##### ✅ Insight 
O comportamento do consumidor frente ao cupom não depende do perfil do restaurante, o que aumenta o potencial de escalabilidade da campanha.




#### Conclusão Geral
A análise segmentada reforça que a campanha de cupom foi amplamente eficaz, com impactos positivos e estatisticamente significativos nos principais KPIs (gasto, frequência e retenção) em todos os segmentos de valor e perfil de consumo.

A campanha gerou aumento consistente de ~13% em pedidos e gasto total por usuário e um aumento expressivo de ~21% na retenção, mesmo após o uso do cupom.

Esses efeitos foram transversais a diferentes tipos de usuário, evidenciando que a ação tem alto potencial de impacto em larga escala.

A única limitação observada foi entre usuários de baixa frequência, onde não houve estímulo à recorrência